# 개요

![](./img1.png)

이번 크롤링 과제로 제가 즐겨하는 게임인 'Teamfight Tactics'(TFT)와 관련된 통계정보를 분석하는 것을 주제로 선정했습니다.  
TFT의 기본적인 플레이 규칙은 아래와 같습니다.  
- 챔피언은 1성, 2성, 3성 3종류의 별 등급이 있다. 기본적으로는 1성만 얻을 수 있으며, 2성은 1성 3개를 합쳐서, 3성은 2성 3개(1성 챔피언 9개)를 합쳐서 만들 수 있다.
    - 챔피언은 1단계, 2단계, 3단계, 4단계, 5단계 챔피언이 있으며, 단계(코스트)가 높을수록 강하지만 비싸고 희귀하다.  
    게임 초반에는 낮은 단계의 챔피언만 볼수있지만 진행될수록 높은 단계의 챔피언이 등장한다.  
    하지만 상점의 총 재고량은 정해져있기 때문에[1] 낮은 단계의 챔피언이 많이 팔릴수록 높은 단계의 챔피언을 볼 확률이 높아진다.
    - 챔피언 조합은 완전 자동이다. 결투장과 대기석에 세 개가 모이는 즉시 합쳐진다. 예외로 전투에 돌입하면 전투가 종료될 때까지 결투장에 있는 챔피언은 합쳐지지 않는다.
    - 한 번 합쳐진 챔피언은 다시 분리할 수 없으며, 각 챔피언이 갖고 있던 아이템도 자동으로 조합된다.
    - 등급이 올라가면 챔피언의 크기가 커지며, 색이 은색/금색으로 변한다. 체력과 공격력도 1.8배씩 높아진다. 소수점은 반올림.
    - 스킬 성능의 경우 1성/2성은 큰 차이가 없지만, 3성이 되면 상당히 차이가 난다.  
    3성 4단계 챔피언은 단독으로도 게임의 판도를 뒤집어버릴만한 성능이고, 3성 5단계는 사실상 승리 확정이다.
    - 1코스트 유닛을 제외한 2성/3성 챔피언은 들인 비용에서 1골드를 뺀 가격으로 팔 수 있다.  
    가령 3코 2성 챔피언은 만드는데 총 9골드(3×3)를 소모되지만, 판매할 때는 8골드인 것. 1코 챔피언들은 반드시 제 값에 팔 수 있다.
- 결투장에 놓을 수 있는 챔피언의 수는 꼬마 전설이(플레이어)의 레벨과 같다. 최대 9레벨. 추가로 대자연의 힘 아이템 한 개당 인원 수를 1명 늘릴 수 있다.
- 전투가 시작되면 모든 챔피언은 AI에 의해 자동으로 움직이며, 플레이어가 컨트롤 할 수 없다. 챔피언들은 적이 자신의 공격 사거리에 들어오도록 이동한다.
    - 근접 챔피언은 원거리 챔피언보다 먼저 움직인다.
    - 일반적으로 모든 챔피언은 한 번 정한 공격 대상을 바꾸지 않는다. 대상이 죽거나, 대상 비지정 상태가 되거나, 최대 사거리에서 1칸 이상 벗어난 상태가 되면 공격대상을 변경한다.
- 결투장과 대기석이 가득찬 상태에서는 챔피언을 구매할 수 없다. 아이템도 창고가 가득 차면 수수께끼 상자를 열 수 없게 된다.

TFT는 현재 [롤체지지](https://lolchess.gg)라는 사이트에서 관련 통계정보를 제공하고 있으며 '추천메타', '천상계 덱'(상위권 플레이어), '아이템 통계'와 같은 정보를 제공하고 있습니다.  
롤체지지 사이트를 크롤링하여 현재 버전에서 플레이어들이 가장 많이 활용하는 메타를 분석하고 사이트에서 제공하는 정보의 가치를 확인해 볼 것입니다.

# Import Library

In [1]:
from urllib.request import urlopen
from urllib.request import Request
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import re
import warnings
import platform
from matplotlib import font_manager, rc

from tqdm.notebook import tqdm

In [2]:
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)
else:
    rc('font', family='AppleGothic')
    
tqdm.pandas()
warnings.filterwarnings('ignore')

# Making Modules

In [3]:
def str_parser(item, type="name"):
    if type == "name":
        ret = item
        ret = re.sub(r'[a-zA-Z]', '', item.get_text()).strip()
    elif type == "trait":
        ret = dict()

        for i in item.select('div'):
            synergy = i['data-tooltip-url'].split('/')[6].split("?")[0].lower()
            stack = i['data-tooltip-url'].split('?')[1].split('=')[1]
            ret[synergy] = int(stack)

    elif type == "high_deck_name":
        synergy_list = item.select('b') + item.select('span')
        tmp = [s.get_text() for s in synergy_list]

        ret =' '.join(tmp)
    elif type == "high_deck_synergy":
        tmp = item.select('b') + item.select('span')
        ret = dict()

        for i in tmp:
            txt = i.get_text().replace('요들 군주', '요들군주').replace('비전 마법사', '비전마법사').replace('범죄 조직', '범죄조직')
            txt = txt.split(' ')
            count = int(txt[0])
            name = txt[1]

            if name == '요들군주': name = '요들 군주'
            if name == '비전마법사': name = '비전 마법사'
            if name == '범죄조직': name = '범죄 조직'

            ret[name] = count
    elif type == "high_unit_name":
        names = item.findAll('div', {'class':'unit__name'})
        ret = []

        for name in names:
            ret.append(name.get_text())
    elif type == "high_stat":
        titles = [i.get_text() for i in item.select('dt')]
        contents = [float(re.sub(r'[^0-9.]', '', i.get_text())) for i in item.select('dd')]

        ret = dict(zip(titles, contents))

    return ret

`BeautifulSoup`을 활용한 데이터를 파싱하는 함수입니다. 넘겨주는 데이터의 type에 맞춰 알맞은 return 값을 파싱해줍니다.  

In [4]:
def get_synergy_list():
    url = "https://lolchess.gg/synergies/set6?hl=ko-KR"
    req = Request(url)
    response = urlopen(req)
    page = response.read()

    bsobj = BeautifulSoup(page, "html.parser")
    synergies_eng = bsobj.findAll('div', {
        'class': re.compile('guide-synergy-table__synergy guide-synergy-table__synergy--[a-z]*')})

    synergies_dict = dict()

    for synergy in synergies_eng:
        eng = synergy['class'][1].split('--')[1]
        kor = synergy.img['alt']
        synergies_dict[eng] = kor

    return synergies_dict

롤체지지 사이트에서 제공하는 추천메타에 나타나는 시너지를 크롤링하는 함수입니다.  
**시너지**란 챔피언들에 부여된 특성들입니다. 공통된 특성을 가진 챔피언들이 일정 인원이상 존재하면 해당 시너지가 발동되며 추가적인 효과를 얻을 수 있습니다.  
시너지는 게임에서 핵심적인 플레이 요소이며 1순위 전략 요소로 사용됩니다.  
따라서 유저의 메타분석과 현재 플레이 트렌드를 분석하는 데에 핵심적인 데이터입니다.

In [5]:
def get_champion_info(champion_list):
    ret = dict()

    for item in champion_list.findAll('div', {'class': re.compile('tft-champion cost-[0-9]*')}):
        item = item.img
        name = item['alt']
        image = item['src']
        ret[name] = image

    return ret

In [6]:
def get_champion_info_url():
    ret = []
    url = 'https://lolchess.gg/champions/set6/zyra?hl=ko-KR'
    req = Request(url)
    response = urlopen(req)
    page = response.read()
    bsobj = BeautifulSoup(page, 'html.parser')
    items = bsobj.findAll('a', {'class': 'guide-champion-list__item'})
    krname = bsobj.findAll('span', {'class':'guide-champion-list__item__name'})
    
    for item, kr in zip(items, krname):
        ret.append((item['href'], kr.getText()))
        
    return ret

def get_champion_base_info():
    ret = dict()
    urls = get_champion_info_url()
    
    for url, data in tqdm(urls):
        into = dict()
        req = Request(url+'?hl=ko-KR')
        response = urlopen(req)
        page = response.read()
        bsobj = BeautifulSoup(page, 'html.parser')
        item = bsobj.find('div', {'class':'guide-champion-detail__stats__value'})
        stats = bsobj.findAll('div', {'class':'guide-champion-detail__stats__value'})
        
        tmp = []
        
        if data != '베이가':
            for stat in stats:
                if len(stat.findAll('img')) == 2:
                    tmp.append(stat.findAll('img')[0]['alt'])
                    tmp.append(stat.findAll('img')[1]['alt'])
                elif stat.img['alt'] != 'Gold':
                    tmp.append(stat.img['alt'])
        else:
            tmp.append('요들 군주')

        name = data
        cost = int(item.getText().strip()[0])
        ret[name] = {"synergy":tmp, "cost":cost}
    
    return ret

메타에 등장하는 챔피언의 정보를 수집하는 함수입니다.  
추가로 챔피언의 가격 및 시너지들을 수집하는 함수입니다.

In [7]:
def get_meta():
    url = 'https://lolchess.gg/meta?hl=ko-KR'
    req = Request(url)
    response = urlopen(req)
    page = response.read()
    bsobj = BeautifulSoup(page, 'html.parser')
    meta_list = list()

    deck_names = bsobj.findAll('div', {'class': 'guide-meta__deck__column name mr-3'})
    traits = bsobj.findAll('div', {'class': 'guide-meta__deck__column traits'})
    champions = bsobj.findAll('div', {'class': 'guide-meta__deck__column champions mr-2'})
    golds = bsobj.findAll('div', {'class': 'guide-meta__deck__column cost mr-2'})

    for deck_name, trait, champion, gold in zip(deck_names, traits, champions, golds):
        deck = dict()

        name = str_parser(deck_name, "name")
        trait_list = str_parser(trait, "trait")
        champion_list = get_champion_info(champion)

        deck['name'] = name
        deck['traits'] = trait_list
        deck['champions'] = list(champion_list.keys())
        deck['gold'] = int(gold.span.get_text())

        meta_list.append(deck)

    return meta_list

문자열 파싱함수, 정보 수집 함수들을 활용하여 '롤체지지'사이트에서 제공하는 '추천메타'정보를 수집하는 함수입니다.  
반환값은 '추천메타'페이지에서 제공되는 조합의 종류에 대한 정보입니다.  
각 조합은 dictionary 자료구조로 저장됩니다.  
- `name`: 조합의 대표 명칭
- `traits`: 조합에서 발동되는 시너지의 종류
- `champions`: 조합에 사용되는 챔피언의 종류
- `gold`: 조합에서 사용되는 모든 챔피언이 2성이 될 때 필요한 가장 이상적인 골드양

In [8]:
def get_high_tier_deck():
    ser = Service("./chromedriver")
    driver = webdriver.Chrome(service=ser)
    driver.implicitly_wait(3)

    url = 'https://lolchess.gg/decks?hl=ko-KR'
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    deck_list = []

    deck_names = soup.findAll('h4', {'class':'deck__traits__name'})
    deck_units = soup.findAll('div', {'class':'deck__units'})
    deck_stats = soup.findAll('div', {'class':'deck__stats'})


    for name, unit, stat in zip(deck_names, deck_units, deck_stats):
        # deck name is deck synergy list
        deck = dict()
        deck_name = str_parser(name, "high_deck_name")
        deck_synergy = str_parser(name, "high_deck_synergy")
        deck_unit = str_parser(unit, "high_unit_name")
        deck_stat = str_parser(stat, "high_stat")
        deck['name'] = deck_name
        deck['units'] = deck_unit
        deck['synergy'] = deck_synergy
        deck['win_rate'] = deck_stat['승률']
        deck['top4_rate'] = deck_stat['TOP4 비율']
        deck['avg_rank'] = deck_stat['평균 등수']

        deck_list.append(deck)

    return deck_list

'롤체지지'사이트에서 제공하는 천상계 유저, 일명 상위권 유저들이 많이 플레이하는 조합정보를 제공하는 '천상계 덱' 사이트 크롤링 함수힙니다.  
천상계 유저는 TFT 전체 플레이어의 상위 약 2.5% 유저들을 대상으로 수집되는 데이터입니다.  
해당 데이터를 수집한 이유는 실제 사이트에서 제공되는 '추천메타'와 상위권 플레이어가 선호하는 메타가 의미가 있는지를 확인하기 위해서입니다.  
해당 페이지는 자바스크립트로 페이지를 로드하기 때문에 셀레니움으로 페이지를 읽고 BeautifuoSoup으로 전처리하였습니다.

In [9]:
from datetime import datetime

def save_df(**kwargs):
    today = datetime.today().strftime("%Y%m%d")
    path = './datas/'
    
    for k, v in kwargs.items():
        if k == 'high':
            v.to_csv(path+today+'_high.csv', index=False)
        elif k == 'meta':
            v.to_csv(path+today+'_meta.csv', index=False)

# Data Load

In [29]:
path = './datas/'
meta_deck_df = pd.read_csv(path+'20211130_meta.csv')
high_deck_df = pd.read_csv(path+'20211130_high.csv')

with open(path+'champion.json', 'r') as f:
    champions = json.load(f)

with open(path+'synergies.json', 'r') as f:
    synergies = json.load(f)
    
with open(path+'20211130_meta.json', 'r') as f:
    metas = json.load(f)
    
with open(path+'20211130_high.json', 'r') as f:
    high_deck_info = json.load(f)

# Making Dataframe

In [36]:
high_deck_df = pd.DataFrame(columns=['name', 'synergy','units', 'win_rate', 'top4_rate', 'avg_rank'])

for item in high_deck_info:
    high_deck_df = high_deck_df.append(item, ignore_index=True)

In [37]:
meta_deck_df = pd.DataFrame(columns=['name','traits', 'champions', 'gold'])

for item in metas:
    meta_deck_df = meta_deck_df.append(item, ignore_index=True)
    
meta_deck_df = meta_deck_df.rename(columns={'traits':'synergy', 'champions':'units'})

# EDA

In [38]:
# meta dataframe synergy name change to Korean version name
meta_deck_df['synergy'] = meta_deck_df['synergy'].apply(lambda x: {synergies[k]:v for k, v in x.items()})

In [39]:
# make mata deck synergy list
syn_list = meta_deck_df['synergy'].tolist()

In [40]:
# make high tier player deck synergy list
high_syn_list = high_deck_df['synergy'].tolist()

## meta deck data EDA

사이트에서 추천하고 있는 '추천메타' 데이터를 분석하겠습니다.

추천메타는 롤체지지에서 제공하는 정보에 따르면 '챌린저'유저 (상위 199명)의 자문을 받아 제공되는 데이터입니다.  
우선 추천메타에서 제공하는 데이터에서 가장 많이 등장하는 챔피언의 종류를 확인하겠습니다.  
많이 활용되는 챔피언일수록 게임 플레이에 기여하는 정도가 높다고 판단할 수 있습니다.  
또한 많은 유저들이 추천메타를 참고로 플레이하는 경향이 많기 때문에 대다수의 유저가 플레이하는 메타라 생각할 수 있습니다.

### meta deck champion trend

In [41]:
from collections import Counter
meta_champions = [champ for meta in meta_deck_df['units'].tolist() for champ in meta]

champ_counts = dict(Counter(meta_champions))
champ_counts_df = pd.DataFrame(columns=['name', 'counts', 'cost', 'synergies'])
champ_counts_df['name'] = champ_counts.keys()
champ_counts_df['counts'] = champ_counts.values()

분석의 편의를 위해 카운팅 관련 데이터프레임을 만들겠습니다.

In [42]:
champ_counts_df['cost'] = champ_counts_df.progress_apply(lambda x: int(champions[x['name']]['cost']), axis=1)
champ_counts_df['synergies'] = champ_counts_df.progress_apply(lambda x: champions[x['name']]['synergy'], axis=1)
meta_deck_df['champ_count'] = meta_deck_df['units'].apply(lambda x: len(x))

  0%|          | 0/56 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

In [43]:
champ_counts_df.head()

,name,counts,cost,synergies
0,직스,2,1,"[고물상, 요들, 비전 마법사]"
1,스웨인,3,2,"[제국, 비전 마법사]"
2,말자하,3,3,"[돌연변이, 비전 마법사]"
3,벡스,2,3,"[요들, 비전 마법사]"
4,럭스,3,4,"[아카데미, 비전 마법사]"


In [44]:
champ_counts_df = champ_counts_df.sort_values('counts', ascending=False, ignore_index=True)
print(f'전체 챔피언의 수: {len(champions)}')
print(f'전체 메타 덱의 수: {len(meta_deck_df)}')
print(f'추천메타에 사용된 챔피언의 수: {len(champ_counts_df)}')
unuse = set(champions.keys()) - set(champ_counts_df['name'].values)
print(f'추천메타에 사용되지 않은 챔피언: {list(unuse)}')
max_list = champ_counts_df[champ_counts_df['counts'] == champ_counts_df['counts'].max()].sort_values('name')
min_list = champ_counts_df[champ_counts_df['counts'] == champ_counts_df['counts'].min()].sort_values('name')
print(f'가장 많이({champ_counts_df.counts.max()}) 사용된 챔피언: {max_list.name.values}')
print(f'가장 적게({champ_counts_df.counts.min()}) 사용된 챔피언: {min_list.name.values}')

print()
print(champ_counts_df.describe().T)

전체 챔피언의 수: 59
전체 메타 덱의 수: 18
추천메타에 사용된 챔피언의 수: 56
추천메타에 사용되지 않은 챔피언: ['일라오이', '트위스티드 페이트', '갱플랭크']
가장 많이(9) 사용된 챔피언: ['잔나']
가장 적게(1) 사용된 챔피언: ['다리우스' '룰루' '미스 포츈' '베이가' '뽀삐' '사이온' '신지드' '우르곳' '워윅' '자이라' '자크' '질리언'
 '카밀' '케이틀린' '퀸' '트런들' '트리스타나' '트위치']

        count      mean       std  min  25%  50%  75%  max
counts   56.0  2.482143  1.684054  1.0  1.0  2.0  3.0  9.0
cost     56.0  2.892857  1.370847  1.0  2.0  3.0  4.0  5.0


추천메타에는 총 56개의 챔피언이 사용되고 있으며 전체 챔피언 59개 중 3개의 챔피언만 사용되지 않고 있습니다.  
가장 많이 사용된 챔피언의 사용 횟수는 9번이며 '잔나'입니다.  
가장 적게 사용된 챔피언의 사용 횟수는 1번이며 '다리우스', '룰루', '미스 포츈', '베이가' 등 챔피언들이 있습니다.

챔피언의 가격과 사용횟수가 상관성이 있는 지를 확인하겠습니다.

In [45]:
df = pd.DataFrame(columns=['cost', 'select'])
df['cost'] = [1,2,3,4,5]
for i in range(1, 6):
    df.loc[(df['cost'] == i), 'select'] = champ_counts_df[champ_counts_df.cost == i]['counts'].sum()

In [46]:
total = champ_counts_df['counts'].sum()

for i in range(len(df)):
    print(f'{i+1} 코스트 챔피언들의 메타 기용률 : {round(df.loc[i].select/total,2) * 100}%')

1 코스트 챔피언들의 메타 기용률 : 12.0%
2 코스트 챔피언들의 메타 기용률 : 18.0%
3 코스트 챔피언들의 메타 기용률 : 22.0%
4 코스트 챔피언들의 메타 기용률 : 27.0%
5 코스트 챔피언들의 메타 기용률 : 21.0%


전반적인 메타의 구성에서 4코스트, 3코스트, 5코스트 챔피언이 기용률이 높은 것을 알 수 있습니다.  
코스트가 높을수록 좋은 성능을 갖게되므로 당연한 결과라고 생각할 수 있습니다.  
그렇다면 이번에는 각 코스트별로 가장 많이 기용된 챔피언과 그 횟수를 알아보도록 하겠습니다.

In [47]:
for i in range(1, 6):
    data = champ_counts_df[champ_counts_df['cost'] == i]
    max_val = data['counts'].max()
    max_data = data[data['counts'] == max_val]
    print(f'{i} 코스트에서 가장 많이 기용된 챔피언: {max_data.name.values}, 기용횟수: {max_val}회')

1 코스트에서 가장 많이 기용된 챔피언: ['카사딘'], 기용횟수: 3회
2 코스트에서 가장 많이 기용된 챔피언: ['블리츠크랭크' '바이'], 기용횟수: 4회
3 코스트에서 가장 많이 기용된 챔피언: ['레오나'], 기용횟수: 5회
4 코스트에서 가장 많이 기용된 챔피언: ['잔나'], 기용횟수: 9회
5 코스트에서 가장 많이 기용된 챔피언: ['유미'], 기용횟수: 8회


### meta deck synergy trend

이번에는 메타덱에 사용되는 시너지를 확인하겠습니다.  
앞서 간단하게 언급했던 것처럼 시너지는 게임의 핵심요소입니다.  
우선 가장 많이 활용된 시너지를 확인하겠습니다.

In [48]:
meta_synergy = meta_deck_df['synergy'].tolist()

In [49]:
synergy_counts = dict(Counter([k for meta in meta_synergy for k in meta.keys()]))
synergy_counts_df = pd.DataFrame(columns=['synergy', 'counts'])
synergy_counts_df['synergy'] = synergy_counts.keys()
synergy_counts_df['counts'] = synergy_counts.values()
synergy_counts_df = synergy_counts_df.sort_values('counts', ascending=False, ignore_index=True)

In [50]:
synergy_counts_df.head()

,synergy,counts
0,학자,9
1,아카데미,8
2,포근,8
3,고물상,7
4,강화술사,6


In [51]:
print(f'전체 시너지의 수: {len(synergies)}')
print(f'전체 메타 덱의 수: {len(meta_deck_df)}')
print(f'추천메타에 사용된 시너지 수: {len(synergy_counts_df)}')
unuse = set(synergies.values()) - set(synergy_counts_df['synergy'].values)
print(f'추천메타에 사용되지 않은 시너지: {list(unuse)}')
max_list = synergy_counts_df[synergy_counts_df['counts'] == synergy_counts_df['counts'].max()].sort_values('synergy')
min_list = synergy_counts_df[synergy_counts_df['counts'] == synergy_counts_df['counts'].min()].sort_values('synergy')
print(f'가장 많이({synergy_counts_df.counts.max()}) 사용된 시너지: {max_list.synergy.values}')
print(f'가장 적게({synergy_counts_df.counts.min()}) 사용된 시너지: {min_list.synergy.values}')

print()
print(synergy_counts_df.describe().T)

전체 시너지의 수: 28
전체 메타 덱의 수: 18
추천메타에 사용된 시너지 수: 27
추천메타에 사용되지 않은 시너지: ['용병']
가장 많이(9) 사용된 시너지: ['학자']
가장 적게(1) 사용된 시너지: ['범죄 조직' '요들' '요들 군주' '화학공학']

        count      mean       std  min  25%  50%  75%  max
counts   27.0  3.814815  2.434322  1.0  2.0  3.0  6.0  9.0


전체 시너지는 28개입니다.  
이때 27개의 시너지가 추천메타에 등장하였으며 사용되지 않은 시너지는 '용병'시너지입니다.  
- 최다 사용 시너지  
'학자' 시너지가 가장 많이 사용된 것으로 보입니다.  
'학자'시너지는 아군 전체의 마나를 회복하는 특성을 갖고 있기 때문에 아군의 스킬 회전율을 높이는 효과입니다.  
스킬 회전율이 높으면 상대에게 동일 시간에 높은 비율의 데미지를 줄 수 있기 때문에 기용률이 높은 것으로 보입니다.  
- 최소 사용 시너지  
가장 적게 카운팅된 시너지는 '범죄 조직', '요들', '요들 군주', '화학공학'임을 알 수 있습니다.  
여기서 '요들'과 '요들 군주'의 경우 공존하는 시너지입니다. 따라서 단순히 '요들'로만 보아도 충분한 것으로 보입니다.  
주의깊게 확인해야 할 점은 '요들'은 다른 시너지들과 다르게 다른 시너지와 함께 섞어 사용하기 어려운 시너지입니다.  
따라서 단독으로 쓰일 수 밖에 없는 것으로 보이며 이에 따라 적은 수의 사용이 된 것으로 보입니다.

이번에는 많이 등장하는 시너지의 조합을 확인하겠습니다.  
즉 전체적으로 완성된 것이 아닌 `['비전 마법사','아카데미']`처럼 부분적으로 자주 등장하는 조합을 확인하고자 합니다.  
이를 확인하면 시너지 간의 좋은 효율을 보여주는 것이 무엇인지 알 수 있을 것입니다.

In [52]:
from itertools import combinations, chain

def all_subsets(ss):
    return chain(*map(lambda x: combinations(ss, x), range(2, len(ss)+1)))

In [53]:
meta_synergy_list = [list(meta.keys()) for meta in meta_synergy]
all_syn_comb = []

for syn in meta_synergy_list:
    for comb in all_subsets(syn):
        all_syn_comb.append(tuple(sorted(comb, key=lambda x: x)))

In [54]:
syn_comb_dic = dict()

for syn in set(all_syn_comb):
    syn_comb_dic[syn] = 0
    for meta_syn in meta_synergy_list:
        if len(set(syn) - set(meta_syn)) == 0:
            syn_comb_dic[syn] += 1

In [55]:
syn_comb_df = pd.DataFrame(columns=['syn_comb', 'counts'])
syn_comb_df['syn_comb'] = syn_comb_dic.keys()
syn_comb_df['counts'] = syn_comb_dic.values()

In [56]:
syn_comb_df = syn_comb_df.sort_values('counts', ascending=False, ignore_index=True)
syn_comb_df[:10]

,syn_comb,counts
0,"(포근, 학자)",7
1,"(강화술사, 학자)",6
2,"(경호대, 아카데미)",6
3,"(아카데미, 포근)",5
4,"(고물상, 학자)",5
5,"(아카데미, 학자)",4
6,"(변형술사, 집행자)",4
7,"(강화술사, 포근, 학자)",4
8,"(아카데미, 포근, 학자)",4
9,"(강화술사, 고물상)",4


In [57]:
print(f"유미 시너지: {champions['유미']['synergy']}")
print(f"잔나 시너지: {champions['잔나']['synergy']}")

유미 시너지: ['아카데미', '포근', '학자']
잔나 시너지: ['고물상', '강화술사', '학자']


상위 10개의 가장 많이 사용된 조합입니다.  
가장 많이 사용된 조합은 '포근'과 '학자'입니다. 하지만 이는 챔피언 '유미'가 가진 조합이므로 유미 챔피언 자체가 많이 사용되는 것을 알 수 있습니다.  
'유미'는 5코스트 챔피언 중 8번이나 기용된 챔피언으로 전체 챔피언에선 '잔나'에 이어 2번째로 많이 기용된 챔피언입니다.  
이것으로보아 '유미'챔피언은 그 자체의 성능이 매우 뛰어난 것으로 판단됩니다.  
두번째로 많이 사용된 시너지 조합은 ('경호대'와 '아카데미'), ('강화술사', '학자')입니다.  
여기서 특이한 점은 '잔나'와 '유미'가 많이 사용된다는 것을 시너지를 통해 알 수 있습니다. 자주 등장하는 시너지는 '잔나'와 '유미'의 등장빈도와 비슷한 것을 알 수 있습니다.

## high deck data EDA

천상계 (상위 2.5%) 플레이어들이 자주 기용하는 조합입니다.  
이 데이터와 함께 볼 수 있는 가장 중요하게 바라볼 것은 '과연 추천 메타에 있는 조합은 천상계에서 사용하는 조합과 얼마나 일치할까?'입니다.

### high deck champion trend

In [58]:
from collections import Counter
meta_champions = [champ for meta in meta_deck_df['units'].tolist() for champ in meta]

champ_counts = dict(Counter(meta_champions))
champ_counts_df = pd.DataFrame(columns=['name', 'counts', 'cost', 'synergies'])
champ_counts_df['name'] = champ_counts.keys()
champ_counts_df['counts'] = champ_counts.values()

In [59]:
high_champions = [champ for meta in high_deck_df['units'].tolist() for champ in meta]

high_counts = dict(Counter(high_champions))
high_counts_df = pd.DataFrame(columns=['name', 'counts', 'cost', 'synergies'])
high_counts_df['name'] = high_counts.keys()
high_counts_df['counts'] = high_counts.values()

high_counts_df['cost'] = high_counts_df.progress_apply(lambda x: int(champions[x['name']]['cost']), axis=1)
high_counts_df['synergies'] = high_counts_df.progress_apply(lambda x: champions[x['name']]['synergy'], axis=1)
high_counts_df = high_counts_df.sort_values('counts', ascending=False, ignore_index=True)

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

In [60]:
high_counts_df.head()

,name,counts,cost,synergies
0,잔나,17,4,"[고물상, 강화술사, 학자]"
1,유미,13,5,"[아카데미, 포근, 학자]"
2,블리츠크랭크,9,2,"[고물상, 경호대, 봉쇄자]"
3,타릭,9,3,"[사교계, 강화술사]"
4,제이스,9,5,"[집행자, 변형술사, 혁신가]"


In [61]:
print(f'전체 챔피언의 수: {len(champions)}')
print(f'전체 상위권 메타 덱의 수: {len(high_deck_df)}')
print(f'상위권 메타에 사용된 챔피언의 수: {len(high_counts_df)}')
unuse = set(champions.keys()) - set(high_counts_df['name'].values)
print(f'상위권 메타에 사용되지 않은 챔피언: {list(unuse)}')
max_list = high_counts_df[high_counts_df['counts'] == high_counts_df['counts'].max()].sort_values('name')
min_list = high_counts_df[high_counts_df['counts'] == high_counts_df['counts'].min()].sort_values('name')
print(f'가장 많이({high_counts_df.counts.max()}) 사용된 챔피언: {max_list.name.values}')
print(f'가장 적게({high_counts_df.counts.min()}) 사용된 챔피언: {min_list.name.values}')

print()
print(high_counts_df.describe().T)

전체 챔피언의 수: 59
전체 상위권 메타 덱의 수: 30
상위권 메타에 사용된 챔피언의 수: 55
상위권 메타에 사용되지 않은 챔피언: ['일라오이', '갱플랭크', '다리우스', '자이라']
가장 많이(17) 사용된 챔피언: ['잔나']
가장 적게(1) 사용된 챔피언: ['가렌' '갈리오' '그레이브즈' '리산드라' '미스 포츈' '사이온' '우르곳' '자크' '코그모' '탐 켄치'
 '트위스티드 페이트' '트위치']

        count      mean       std  min  25%  50%  75%   max
counts   55.0  4.436364  3.365201  1.0  2.0  3.0  6.5  17.0
cost     55.0  2.909091  1.378038  1.0  2.0  3.0  4.0   5.0


추천메타에 비해 12개 더 많은 조합이 존재합니다.  
하지만 오히려 사용되지 않은 챔피언의 수는 증가한 것을 알 수 있습니다.
특이한 점은 가장 많이 사용된 챔피언이 '잔나'인 것은 변하지 않는 것을 알 수 있습니다.  
대신 오히려 적게 사용된 챔피언의 종류가 줄어든 것으로 보아 상위권은 조합의 다양성이 증가한 것을 알 수 있습니다.

In [62]:
"""
추천 메타의 코스트 기용률
1 코스트 챔피언들의 메타 기용률 : 12.0%
2 코스트 챔피언들의 메타 기용률 : 18.0%
3 코스트 챔피언들의 메타 기용률 : 22.0%
4 코스트 챔피언들의 메타 기용률 : 27.0%
5 코스트 챔피언들의 메타 기용률 : 21.0%
"""

df = pd.DataFrame(columns=['cost', 'select'])
df['cost'] = [1,2,3,4,5]
for i in range(1, 6):
    df.loc[(df['cost'] == i), 'select'] = high_counts_df[champ_counts_df.cost == i]['counts'].sum()

total = high_counts_df['counts'].sum()

for i in range(len(df)):
    print(f'{i+1} 코스트 챔피언들의 메타 기용률 : {round(df.loc[i].select/total,2) * 100}%')

1 코스트 챔피언들의 메타 기용률 : 0.0%
2 코스트 챔피언들의 메타 기용률 : 0.0%
3 코스트 챔피언들의 메타 기용률 : 0.0%
4 코스트 챔피언들의 메타 기용률 : 0.0%
5 코스트 챔피언들의 메타 기용률 : 0.0%


![](./cost.png)

추천메타와 유사하게 3, 4, 5코스트 챔피언의 기용률이 높은 것을 알 수 있습니다.  
하지만 눈에 띄게 특이한 것은 4코스트 챔피언들의 기용률이 증가하였고 5코스트 챔피언의 기용률이 감소한 것을 알 수 있습니다.  
이는 5코스트 챔피언의 확률과 관련이 있는 것으로 보이는데 5코스트 (그림의 티어 5) 챔피언의 최대 확률을 15%입니다.  
따라서 얻기 힘든 확률의 5코스트에 투자하는 것보단 좀 더 높은 확률의 4코스트 (그림의 티어 4)에 투자를 하는 것을 선호하는 것으로 보입니다.

이번에는 천상계에서 각 코스트별 어떤 챔피언은 많이 기용했는지 확인하겠습니다.

In [63]:
"""
추천 메타 데이터
1 코스트에서 가장 많이 기용된 챔피언: ['카사딘'], 기용횟수: 3회
2 코스트에서 가장 많이 기용된 챔피언: ['블리츠크랭크' '바이'], 기용횟수: 4회
3 코스트에서 가장 많이 기용된 챔피언: ['레오나'], 기용횟수: 5회
4 코스트에서 가장 많이 기용된 챔피언: ['잔나'], 기용횟수: 9회
5 코스트에서 가장 많이 기용된 챔피언: ['유미'], 기용횟수: 8회
"""

for i in range(1, 6):
    data = high_counts_df[high_counts_df['cost'] == i]
    max_val = data['counts'].max()
    max_data = data[data['counts'] == max_val]
    print(f'{i} 코스트에서 가장 많이 기용된 챔피언: {max_data.name.values}, 기용횟수: {max_val}회')

1 코스트에서 가장 많이 기용된 챔피언: ['이즈리얼'], 기용횟수: 7회
2 코스트에서 가장 많이 기용된 챔피언: ['블리츠크랭크'], 기용횟수: 9회
3 코스트에서 가장 많이 기용된 챔피언: ['타릭' '레오나'], 기용횟수: 9회
4 코스트에서 가장 많이 기용된 챔피언: ['잔나'], 기용횟수: 17회
5 코스트에서 가장 많이 기용된 챔피언: ['유미'], 기용횟수: 13회


4코스트와 5코스트에서 가장 많이 기용된 챔피언은 '잔나'와 '유미'로 동일한 것을 알 수 있습니다.  
또한 2, 3코스트의 기용 챔피언도 유사한 경향을 보이는 것을 알 수 있습니다.

### high deck synergy trend

이번에는 천상계덱에 사용되는 시너지를 확인하겠습니다.  
우선 가장 많이 활용된 시너지를 확인하겠습니다.

In [64]:
high_synergy = high_deck_df['synergy'].tolist()

In [65]:
high_synergy_counts = dict(Counter([k for meta in high_synergy for k in meta.keys()]))
high_synergy_counts_df = pd.DataFrame(columns=['synergy', 'counts'])
high_synergy_counts_df['synergy'] = high_synergy_counts.keys()
high_synergy_counts_df['counts'] = high_synergy_counts.values()
high_synergy_counts_df = high_synergy_counts_df.sort_values('counts', ascending=False, ignore_index=True)

In [66]:
high_synergy_counts_df.head()

,synergy,counts
0,고물상,18
1,아카데미,18
2,학자,16
3,사교계,14
4,포근,13


In [67]:
print(f'전체 시너지의 수: {len(synergies)}')
print(f'전체 상위권 메타 덱의 수: {len(high_deck_df)}')
print(f'상위권 메타에 사용된 시너지 수: {len(high_synergy_counts_df)}')
unuse = set(synergies.values()) - set(high_synergy_counts_df['synergy'].values)
print(f'상위권 메타에 사용되지 않은 시너지: {list(unuse)}')
max_list = high_synergy_counts_df[high_synergy_counts_df['counts'] == high_synergy_counts_df['counts'].max()].sort_values('synergy')
min_list = high_synergy_counts_df[high_synergy_counts_df['counts'] == high_synergy_counts_df['counts'].min()].sort_values('synergy')
print(f'가장 많이({high_synergy_counts_df.counts.max()}) 사용된 시너지: {max_list.synergy.values}')
print(f'가장 적게({high_synergy_counts_df.counts.min()}) 사용된 시너지: {min_list.synergy.values}')

print()
print(high_synergy_counts_df.describe().T)

전체 시너지의 수: 28
전체 상위권 메타 덱의 수: 30
상위권 메타에 사용된 시너지 수: 26
상위권 메타에 사용되지 않은 시너지: ['제국', '용병']
가장 많이(18) 사용된 시너지: ['고물상' '아카데미']
가장 적게(1) 사용된 시너지: ['대식가' '범죄 조직' '화학공학']

        count      mean       std  min  25%  50%   75%   max
counts   26.0  6.653846  5.505941  1.0  2.0  5.0  10.5  18.0


추천 메타의 결과와 다르게 '고물상'시너지가 가장 많이 사용된 것을 알 수 있습니다.  
'고물상' 시너지의 효과는 장착 아이템이 완성 아이템이 아닌 재료인 경우 해당 아이템을 완성 아이템으로 만들어줍니다.  
이를 토대로 보면 상위권에서는 시너지 자체의 파워보다 아이템의 보유량이 영향을 많이 미치는 것으로 판단할 수도 있습니다.  
하지만 여기서 주목해야할 것은 '잔나'의 시너지입니다.  
상위권에서는 '잔나' 챔피언이 압도적으로 많이 기용됩니다. '잔나'의 시너지 중 하나가 '고물상'인만큼 '고물상'이 사용될 확률이 높을 것입니다.  
하지만 '고물상'은 '고물상' 챔피언이 2개 이상 공존해야 발동되므로 이외에 같이 사용되는 챔피언이 있을 것으로 보입니다.

추천 메타와 마찬가지로 가장 많이 사용된 부분 조합을 확인하겠습니다.

In [68]:
high_synergy_list = [list(meta.keys()) for meta in high_synergy]
all_syn_comb = []

for syn in high_synergy_list:
    for comb in all_subsets(syn):
        all_syn_comb.append(tuple(sorted(comb, key=lambda x: x)))

In [69]:
syn_comb_dic = dict()

for syn in set(all_syn_comb):
    syn_comb_dic[syn] = 0
    for meta_syn in high_synergy_list:
        if len(set(syn) - set(meta_syn)) == 0:
            syn_comb_dic[syn] += 1

In [70]:
syn_comb_df = pd.DataFrame(columns=['syn_comb', 'counts'])
syn_comb_df['syn_comb'] = syn_comb_dic.keys()
syn_comb_df['counts'] = syn_comb_dic.values()

In [71]:
"""
    syn_comb	counts
0	(포근, 학자)	7
1	(경호대, 아카데미)	6
2	(강화술사, 학자)	6
3	(아카데미, 포근)	5
4	(고물상, 학자)	5
5	(아카데미, 학자)	4
6	(난동꾼, 집행자)	4
7	(강화술사, 포근)	4
8	(강화술사, 시계태엽)	4
9	(강화술사, 포근, 학자)	4
"""
syn_comb_df = syn_comb_df.sort_values('counts', ascending=False, ignore_index=True)
syn_comb_df[:10]

,syn_comb,counts
0,"(강화술사, 학자)",11
1,"(아카데미, 포근)",11
2,"(고물상, 아카데미)",10
3,"(고물상, 학자)",10
4,"(포근, 학자)",10
5,"(사교계, 학자)",9
6,"(경호대, 아카데미)",9
7,"(고물상, 사교계)",9
8,"(아카데미, 포근, 학자)",8
9,"(강화술사, 사교계)",8


앞서 보았던 추천 메타와 다르게 ('강화술사', '학자')가 높은 비율로 나타나는 것을 알 수 있습니다.  
이는 '잔나'가 가진 시너지의 조합과 일치하며 역시나 '잔나'의 기용이 높은 것으로 보입니다.

### Champions uitlity cost

이번에는 천상계 메타에서 챔피언들의 효용가치를 계산하겠습니다.  
효용가치 식은 아래와 같이 정의하겠습니다.  
$$
\text{효용가치(utility_cost)} = \frac{\text{챔피언의 기용횟수(counts)}}{\text{챔피언의 코스트(cost)}} \times \frac{\text{챔피언 사용 덱의 평균승률}\times \text{챔피언 사용 덱의 평균 top4 비율}}{\text{챔피언이 사용된 덱들의 평균등수의 평균}}
$$  
챔피언이 많이 기용될수록 가치가 높다고 판단하지만 그에 비해 챔피언의 코스트는 투자비용으로 바라볼 수 있습니다.  
또한 해당 챔피언이 포함된 덱들의 평균 승률과 top4비율이 높을수록 승리에 기여한다고 볼 수 있으며 평균등수들의 평균이 낮을수록 높은 가중치를 갖습니다.

In [72]:
champions_df = pd.DataFrame(columns=['name', 'cost'])
champions_df['name'] = champions.keys()
champions_df['cost'] = champions_df['name'].apply(lambda x: champions[x]['cost'])

In [73]:
champions_df = pd.merge(champions_df, high_counts_df[['name', 'counts']], how='outer', on='name').fillna(0)
champions_df.counts = champions_df.counts.astype(int)

In [74]:
def get_avg_data(x):
    """
    x: name
    """
    avg_avg_rank = 0
    avg_win_rate = 0
    avg_top4_rate = 0
    tmp = []
    
    for idx, unit in enumerate(high_deck_df['units']):
        if x in unit:
            tmp.append(idx)
            
    avg_avg_rank = round(high_deck_df.loc[tmp]['avg_rank'].mean(), 2)
    avg_win_rate = round(high_deck_df.loc[tmp]['win_rate'].mean()/100, 2)
    avg_top4_rate = round(high_deck_df.loc[tmp]['top4_rate'].mean()/100, 2)
    return (avg_avg_rank, avg_win_rate, avg_top4_rate)

In [75]:
champions_df = champions_df.sort_values('counts', ascending=False, ignore_index=True)
champions_df['avg_avg_rank'] = champions_df['name'].progress_apply(lambda x: get_avg_data(x)[0])
champions_df['avg_win_rate'] = champions_df['name'].progress_apply(lambda x: get_avg_data(x)[1])
champions_df['avg_top4_rate'] = champions_df['name'].progress_apply(lambda x: get_avg_data(x)[2])

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

In [76]:
champions_df['utility_cost'] = round((champions_df['counts']/champions_df['cost'])\
                                *((champions_df['avg_win_rate'] * champions_df['avg_top4_rate'])/champions_df['avg_avg_rank']),5)

In [77]:
champions_df = champions_df.sort_values('utility_cost', ascending=False, ignore_index=True)
champions_df.describe()

,cost,counts,avg_avg_rank,avg_win_rate,avg_top4_rate,utility_cost
count,59.000000,59.000000,55.000000,55.000000,55.000000,55.000000
mean,2.830508,4.135593,2.856182,0.290000,0.812545,0.155327
std,1.379062,3.436406,0.497659,0.073384,0.096268,0.132067
min,1.000000,0.000000,2.080000,0.160000,0.500000,0.004560
25%,2.000000,1.000000,2.540000,0.240000,0.775000,0.061275
50%,3.000000,3.000000,2.810000,0.280000,0.830000,0.128930
75%,4.000000,6.000000,3.020000,0.350000,0.860000,0.204120
max,5.000000,17.000000,4.370000,0.460000,0.940000,0.709570


In [78]:
champions_df[champions_df['counts']>4].head(10)

,name,cost,counts,avg_avg_rank,avg_win_rate,avg_top4_rate,utility_cost
0,직스,1,6,2.53,0.34,0.88,0.70957
1,이즈리얼,1,7,3.01,0.27,0.77,0.48349
3,신지드,1,5,3.03,0.28,0.76,0.35116
4,잔나,4,17,2.86,0.28,0.81,0.33703
5,블리츠크랭크,2,9,2.84,0.24,0.83,0.31563
7,벡스,3,8,2.61,0.32,0.87,0.28444
8,스웨인,2,6,2.73,0.30,0.85,0.28022
9,타릭,3,9,2.73,0.29,0.84,0.26769
10,카타리나,2,6,2.78,0.24,0.85,0.22014
11,탈론,2,6,2.78,0.24,0.85,0.22014


챔피언의 효용가치를 계산한 결과입니다.  
챔피언의 기용횟수 평균인 4회보다 적은 챔피언들의 표본은 제외한 결과입니다.  
가장 높은 효용가치를 가진 챔피언은 직스이고 0.71238의 가치를 갖습니다.   
효용가치가 높은 챔피언들은 대부분 각 지표에서 평균보다 좋은 통계치를 갖는 것을 알 수 있습니다.  
특이한 점은 '이즈리얼'챔피언은 전반적으로 평균 통계치들보다 낮지만 높은 효용가치를 갖고 있습니다.  
효용가치 상위권에 '잔나'가 포함된 것으로 보아 '잔나' 챔피언의 가치는 굉장히 높은 것으로 보입니다.  
4코스트 챔피언들을 많이 기용한 것에 비해 3코스트와 2코스트 챔피언의 효용가치가 높은 것으로 보이며 이는 4코스트 챔피언 중 실제 사용하는 챔피언의 수가 치우쳐져 있다고 생각할 수 있습니다.

## high deck and meta deck

앞서 언급한 추천메타의 정보는 과연 상위권의 선택과 얼마나 일치할까를 확인할 것입니다.  

In [79]:
meta_synergy = meta_deck_df['synergy'].values.tolist()
meta_synergies = []

for meta in meta_synergy:
    tmp = [str(v)+' '+str(k) for k, v in meta.items()]
    meta_synergies.append(tmp)
    
high_synergy = high_deck_df['synergy'].values.tolist()
high_synergies = []

for high in high_synergy:
    tmp = [str(v)+' '+str(k) for k, v in high.items()]
    high_synergies.append(tmp)

In [80]:
cnt = 0
exist = dict()
for meta in meta_synergies:
    for idx, high in enumerate(high_synergies):
        if len(set(meta) - set(high)) == 0:
            m = ', '.join(meta)
            if m not in exist:
                tmp = {"win":[], "top4":[], "rank":[]}
                exist[m] = tmp
            win, rank, top4 = high_deck_df.loc[idx][['win_rate','avg_rank','top4_rate']]
            exist[m]['win'].append(win)
            exist[m]['top4'].append(top4)
            exist[m]['rank'].append(rank)

            cnt+=1

In [81]:
print(f"추천 메타의 발전된 형태의 천상계 덱 수: {cnt}")
print(f"추천 메타의 발전된 형태의 천상계 덱 수 비율: {round(cnt/len(meta_deck_df),2)}")

추천 메타의 발전된 형태의 천상계 덱 수: 11
추천 메타의 발전된 형태의 천상계 덱 수 비율: 0.61


추천 메타에서 제공한 메타 중 약 67%가 상위권에서 응용한 덱으로 사용되는 것을 알 수 있습니다.  
표본 자체가 많지 않은 상황에서 이 정도의 비율이 사용되면 충분히 높은 것으로 볼 수 있는 것으로 판단됩니다.

In [82]:
exist_df = pd.DataFrame(columns=['meta_name', 'win_rate', 'top4_rate', 'avg_rank'])
exist_df['meta_name'] = exist.keys()
exist_df['win_rate'] = exist_df['meta_name'].progress_apply(lambda x: round(np.mean(exist[x]['win']), 2))
exist_df['top4_rate'] = exist_df['meta_name'].progress_apply(lambda x: round(np.mean(exist[x]['top4']), 2))
exist_df['avg_rank'] = exist_df['meta_name'].progress_apply(lambda x: round(np.mean(exist[x]['rank']), 2))

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [83]:
meta_deck_df['syn_name'] = meta_synergies
meta_deck_df['syn_name'] = meta_deck_df['syn_name'].apply(lambda x: ', '.join(x))

In [84]:
exist_df['name'] = exist_df['meta_name'].apply(lambda x: meta_deck_df[meta_deck_df['syn_name'] == x]['name'].values[0])

In [85]:
exist_df

,meta_name,win_rate,top4_rate,avg_rank,name
0,"6 비전 마법사, 1 포근, 2 아카데미, 2 고물상, 2 학자",30.17,84.13,2.74,비전마법사 럭스
1,"2 자매, 1 대식가, 5 화학공학, 4 난동꾼, 2 쌍발총",41.80,92.80,2.17,난동꾼 우르곳
2,"1 포근, 3 강화술사, 2 아카데미, 2 시계태엽, 2 경호대, 2 학자, 2 저...",16.70,67.30,3.62,진 브라움
3,"6 고물상, 2 자매, 1 변형술사, 2 집행자, 2 난동꾼",22.20,79.30,3.00,6 고물상 트런들
4,"5 돌연변이, 2 거신, 2 난동꾼, 1 사교계",16.20,57.60,4.07,돌연변이 초가스 말자하
5,"3 봉쇄자, 3 돌연변이, 2 아카데미, 2 집행자, 2 쌍발총, 2 난동꾼, 2 저격수",24.00,81.20,2.97,가렌 코그모
6,"6 암살자, 2 아카데미, 2 고물상, 2 경호대",21.40,78.70,3.07,6 암살자 카타리나
7,"1 요들 군주, 6 요들, 2 고물상, 2 비전 마법사, 2 강화술사, 2 학자",38.30,92.75,2.26,요들군주 베이가


추가로 천상계에서 사용된 추천메타 덱 리스트입니다.  
가장 높은 승률을 보이는 덱은 '난동꾼 우르곳'입니다.